In [5]:
import os
print(f"Current working directory: {os.getcwd()}")
os.chdir("../")

Current working directory: /home/vairavan/projects/Algotrader/notebooks


In [6]:
import logging
from src.utils.logging_setup import setup_general_logging

setup_general_logging(
    "/home/vairavan/projects/AlgoTrader/logs/",
    level=logging.INFO,
    log_to_console=True,
    log_to_file=False
)
# Verify logging is working
logging.info("Logging setup complete")

2025-05-15 01:44:50 -0500 | INFO     | /home/vairavan/projects/Algotrader/src/utils/logging_setup.py | General logging configured successfully.
2025-05-15 01:44:50 -0500 | INFO     | root | Logging setup complete


In [7]:
from ib_async import util
util.startLoop()  # Start the asyncio event loop
from src.utils.helpers import load_json_config
ibkr_config = load_json_config("config/accounts/ibkr_default.json")

from src.data_management.ibkr_helper import IBKRHelper
timezone = ibkr_config["ibkr"]["trading_timezone"]
ibkr = IBKRHelper(market_timezone_str=timezone)
await ibkr.connect(port=IBKRHelper.TWS_PORT,force_new=True,client_id=36)  # Connect to TWS or IB Gateway

2025-05-15 01:44:53 -0500 | INFO     | src.data_management.ibkr_helper | Successfully connected to IB at 127.0.0.1:7497


True

2025-05-15 01:53:04 -0500 | ERROR    | ib_async.wrapper | Error 162, reqId 250: Historical Market Data Service error message:HMDS query returned no data: NIFTY2552223550PE@NSE Trades, contract: Contract(secType='OPT', conId=778477177, symbol='NIFTY50', lastTradeDateOrContractMonth='20250522', strike=23550.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='NIFTY2552223550PE', tradingClass='NIFTY')
2025-05-15 01:56:10 -0500 | ERROR    | ib_async.wrapper | Error 162, reqId 402: Historical Market Data Service error message:HMDS query returned no data: NIFTY2551525350PE@NSE Trades, contract: Contract(secType='OPT', conId=775823876, symbol='NIFTY50', lastTradeDateOrContractMonth='20250515', strike=25350.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='NIFTY2551525350PE', tradingClass='NIFTY')
2025-05-15 07:22:22 -0500 | ERROR    | ib_async.wrapper | Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
2025-05-

In [12]:
import datetime
from ib_async import Index
from rich.progress import track
from tqdm import tqdm
DATA_DIR = "/home/vairavan/projects/AlgoTrader/data/historical/NIFTY/"

today = datetime.datetime.now(tz=ibkr.market_timezone).replace(hour=0,minute=0,second=0,microsecond=0)
days_since_thursday = (today.weekday() - 3) % 7 + 7  # 3 is Thursday (Monday=0)
last_week_thursday = today - datetime.timedelta(days=days_since_thursday)

all_weekdays = [last_week_thursday + datetime.timedelta(days=i) 
                for i in range(0,(today - last_week_thursday).days+1) 
                if (last_week_thursday + datetime.timedelta(days=i)).weekday() <5]

all_contracts = await ibkr.get_option_chains_new(expiry_weeks=1)
all_contracts = [c for c in all_contracts if c.lastTradeDate == today.strftime("%Y%m%d")]

idx = Index(symbol="NIFTY50", exchange="NSE", currency="INR")
index_history = await ibkr.get_historical_bars(contract=idx, start_date=all_weekdays[0], end_date=all_weekdays[-1], bar_size="1 day", return_bars=True)

for i, bar in enumerate(index_history):

    day = bar.date
    contracts = [c for c in all_contracts if c.strike > index_history[i].close-500 and c.strike < index_history[i].close+500]
    for c in tqdm(contracts, total=len(contracts), desc=f"{i+1}/{len(index_history)}: Processing contracts for {day.date()}", ncols=30):
        data_logger = ibkr.get_bardata_logger(contract=c, bar_size="1 day", data_dir=DATA_DIR)
        hist_bars = await ibkr.get_historical_bars(c, day, day, bar_size="1 day", return_bars=True)
        
        if hist_bars is not None and len(hist_bars) > 0:
            data_logger.log_bars(hist_bars)
            data_logger = ibkr.get_bardata_logger(contract=c, bar_size="5 secs", data_dir=DATA_DIR)
            hist_bars = await ibkr.get_historical_bars(c, day, day, bar_size="5 secs", return_bars=True )
            if hist_bars is not None and len(hist_bars) > 0:
                data_logger.log_bars(hist_bars)


2025-05-15 01:48:33 -0500 | INFO     | src.data_management.ibkr_helper | Retrieved 986 from option chains for symbol=NIFTY50, strike_range= +/- None,expiry month = None and expiry week = 1.


2/5: Processing contracts for 2025-05-09:  51%|█████▏    | 41/80 [01:21<01:09,  1.79s/it]

2025-05-15 01:53:04 -0500 | WARNING  | src.data_management.ibkr_helper | No historical bars received for Contract(secType='OPT', conId=778477177, symbol='NIFTY50', lastTradeDateOrContractMonth='20250522', strike=23550.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='NIFTY2552223550PE', tradingClass='NIFTY')


3/5: Processing contracts for 2025-05-12:  46%|████▋     | 37/80 [01:22<02:02,  2.84s/it]

2025-05-15 01:56:10 -0500 | WARNING  | src.data_management.ibkr_helper | No historical bars received for Contract(secType='OPT', conId=775823876, symbol='NIFTY50', lastTradeDateOrContractMonth='20250515', strike=25350.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='NIFTY2551525350PE', tradingClass='NIFTY')


5/5: Processing contracts for 2025-05-14: 100%|██████████| 80/80 [03:23<00:00,  2.55s/it]
